In [ ]:
# as additional information during development, 
    # Only to get_column_names
    def get_column_names(self, all_columns = False, engin_parameters = True):  # all_columns=False --- doesn't work yet
        if all_columns == False:
            self.indices_col = [i for i in range(1, len(self.df.columns) + 1, 2)]
            self.names_col = [self.df.columns[i] for i in range(1, len(self.df.columns), 2)]
        else:
            self.indices_col = [i for i in range(len(self.df.columns))]
            self.names_col = [self.df.columns[i] for i in range(len(self.df.columns))]
        
        self.n_df_columns = pd.DataFrame(np.array([self.indices_col, self.names_col]).T) 
            
        self.nc = 6
        self.nr = int(np.ceil(len(self.n_df_columns.index) / self.nc))
        self.lst_nr = [i for i in range(self.nr)]
        self.col_gen = (s for s in itertools.cycle(('index', 'Parameters')))
        self.df_cname = pd.DataFrame(np.zeros(self.nr))
        for i in range(self.nc):
            self.df_cname.insert(len(self.df_cname.columns), f'{next(self.col_gen)}_{i}', 
                                 self.n_df_columns.iloc[:self.nr, 0])
            self.df_cname.insert(len(self.df_cname.columns), f'{next(self.col_gen)}_{i}', 
                                 self.n_df_columns.iloc[:self.nr, 1])
            if len(self.n_df_columns.index) >= self.nr:
                self.n_df_columns.drop(self.lst_nr, inplace=True)
                self.n_df_columns.index = [i for i in range(len(self.n_df_columns.index))]
        
        self.df_cname.drop(0, axis=1, inplace=True)
        return self.df_cname 
    
    
    def choose_rowes(self):
        pass

In [ ]:
#def filter_fuel(self): remove data outliers --- выбросы ---   quantile(0.01) - quantile(0.99)
        self.step = 300
        self.indices = []
        for i in range(0, len(self.res_filtered.index), self.step):
            self.lst_range = [j for j in self.res_filtered.index[i:(i+self.step)]]
            self.q_low = self.res_filtered.loc[self.lst_range, 'Zużycie paliwa średnie[g/s]'].quantile(0.01)
            self.q_hi  = self.res_filtered.loc[self.lst_range, 'Zużycie paliwa średnie[g/s]'].quantile(0.99)
            self.index = self.res_filtered[(self.res_filtered['Zużycie paliwa średnie[g/s]'] >= self.q_low) & 
                             (self.res_filtered['Zużycie paliwa średnie[g/s]'] <= self.q_hi)].index
            self.indices += list(self.index)
        self.drop_ind = list(set(self.res_filtered.index) - set(self.indices))
        self.res_filtered.fillna(method='ffill', inplace=True)
        print(f'len(self.drop_ind) - {len(self.drop_ind)}')

In [ ]:
# def filter_torque(self, lst):

self.indices = []
self.gr1 = self.res_filtered.groupby('Torque')  # group by 'Torque'
self.dic1 = dict(list(self.gr1))
for df in self.dic1.values():
    self.q_low = df.loc[:, 'Moment obrotowy[Nm]'].quantile(0.01)
    self.q_hi  = df.loc[:, 'Moment obrotowy[Nm]'].quantile(0.99)
    self.index = df[(df['Moment obrotowy[Nm]'] >= self.q_low) & 
                     (df['Moment obrotowy[Nm]'] <= self.q_hi)].index
    self.indices += list(self.index)
self.drop_ind = list(set(self.res_filtered.index) - set(self.indices))
self.res_filtered.drop(self.drop_ind, inplace=True)
print(f'len(self.drop_ind) - {len(self.drop_ind)}')

In [ ]:
def anal_fuel(min_time_f=500):

    lst_div_f = [0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.8, 1]
    t1.div_f = 2
    lst_f = np.arange(0.05, 4, 0.01)
    lst_min_time_f = [10, 100, 500, 700, 800, 900, 1000, 1500, 2000]
    t1.min_time_f = 2000
    
    t1.filter_data()
    df_f_anal = pd.DataFrame(index=t1.dict_gr_res.keys())
    
    
    ser = pd.Series(dtype='float64')
    ser.name = f'div_f={t1.div_f}--min_time_f={t1.min_time_f}'
    for k in t1.dict_gr_res:
        ser.loc[k] = len(t1.dict_gr_res[k])
    frame = [df_f_anal, ser]
    df_f_anal = pd.concat(frame, axis=1, join='outer')
    
    
    t1.min_time_f = min_time_f
    for div_f in lst_div_f:
        t1.div_f = div_f
        t1.filter_data()
        ser = pd.Series(dtype='float64')
        ser.name = div_f
        for k in t1.dict_gr_res:
            ser.loc[k] = len(t1.dict_gr_res[k])

        frame = [df_f_anal, ser]
        df_f_anal = pd.concat(frame, axis=1, join='outer')

    return df_f_anal
        #lst_val.append(len(t1.dict_gr_res[k])
    #    print(f'{k} --- {len(t1.dict_gr_res[k])}')

#df_f_anal = pd.DataFrame(lst_val, index=t1.dict_gr_res.keys(), columns=lst_min_time_f)
#df_f_anal
anal_fuel(min_time_f=500)

In [ ]:
def filter_fuel(self):
        self.res_filtered.drop(self.res_filtered[self.res_filtered['Zużycie paliwa średnie[g/s]'] < 0.5].index, 
                               inplace=True)
        
        self.indices = []
        for f in self.lst_f:
            self.div_min = f - self.div_f
            self.div_max = f + self.div_f
            self.index = self.res_filtered[(self.res_filtered['Zużycie paliwa średnie[g/s]'] >= self.div_min) & 
                             (self.res_filtered['Zużycie paliwa średnie[g/s]'] <= self.div_max)].index
            if len(self.index) > self.min_time_f:
                self.indices += list(self.index)
                
        self.drop_ind = list(set(self.res_filtered.index) - set(self.indices))
        self.res_filtered.drop(self.drop_ind, inplace=True)
        
        self.dict_all_filtures['def filter_fuel(self):'] = True
        self.dict_all_filtures['    5.amount of rowes --- '] = len(self.res_filtered)

In [ ]:
#self.max_leaf_nodes = max_leaf_nodes
        
        self.model_DT = DecisionTreeRegressor(max_leaf_nodes=self.max_leaf_nodes, 
                                             random_state=0)
        self.model_DT.fit(self.train_X, self.train_y)
        self.val_predicted_DT = self.model_DT.predict(self.val_X)
        self.MAE_DT = mean_absolute_error(self.val_y, self.val_predicted_DT)
        
        self.dict_param['DecisionTree() - '] = {'max_leaf_nodes': self.max_leaf_nodes,
                                                'MAE_DT': round(self.MAE_DT, 4)}

In [ ]:
def choose_max_leaf_nodes(self):  
        self.lst_leaf_nodes = [5, 50, 500, 5000, 50000]
        self.lst_MAEs = []
        for max_leaf_nodes in self.lst_leaf_nodes:
            self.DecisionTree(max_leaf_nodes=max_leaf_nodes)
            self.lst_MAEs.append(self.MAE_DT)
              
        self.dict_nodes_MAEs = dict(zip(self.lst_MAEs, self.lst_leaf_nodes))
        self.df_nodes_MAEs = pd.DataFrame([self.lst_leaf_nodes, self.lst_MAEs],
                                          index=['max_leaf_nodes', 'MAE']).T
        self.leaf_nodes = self.dict_nodes_MAEs[min(self.lst_MAEs)]  # leaf nodes for min MAE
        self.max_leaf_nodes = self.leaf_nodes

In [ ]:
    def RandomForest(self):    
        self.val_predicted_RF = self.model_RF_best.predict(self.val_X)
        
        self.model_RF.fit(self.train_X, self.train_y)
        self.val_predicted_RF = self.model_RF.predict(self.val_X)
        self.MAE_RF = mean_absolute_error(self.val_y, self.val_predicted_RF)
        self.MSE_RF = mean_squared_error(self.val_y, self.val_predicted_RF)
        self.r2_RF = r2_score(self.val_y, self.val_predicted_RF)

In [ ]:
### we use cros validation - train_test_split goes automaticaly
self.train_X, self.val_X, self.train_y, self.val_y = train_test_split(self.X, self.y, 
                                                                      random_state = 0, 
                                                                     train_size=0.8, test_size=0.2)

In [ ]:
''' !!!!!  def XGBoost   !!!!!!!'''
    def XGBoost(self):
        self.model_XGB = XGBRegressor(n_estimators=1000, learning_rate=0.01, n_jobs=5)
        self.model_XGB.fit(X_train_transformed, y_train, 
                           early_stopping_rounds=5, 
                           eval_set=[(X_val_transformed, y_val)], 
                           verbose=False)
        self.val_predicted_XGB = self.model_XGB.predict(X_val_transformed)
        self.MAE_XGB = mean_absolute_error(self.val_y, self.val_predicted_XGB)
        
        self.dict_MAEs['XGBoost() - '] = self.MAE_XGB

In [ ]:
self.dict_anlyses = open(f"dict_anlyses_{self.res_name}.json", "w")
json.dump(self.dict_param, self.dict_anlyses)
self.dict_anlyses.close()

self.dict_anlyses = open(f"dict_anlyses_{self.res_name}.json", "r")
self.output = self.dict_anlyses.read()
print(self.output)
self.output.close()

''' ------------------------------'''
self.models_param = open(f"models_param_{self.res_name}.pkl", "wb")
pickle.dump(self.dict_param, self.models_param)
self.models_param.close()

self.models_param = open(f"models_param_{self.res_name}.pkl", "rb")
self.output = pickle.load(self.models_param)
print(self.output)
self.models_param.close()

In [ ]:
raw_data = {'tr_ON' : ['raw_data\\Zew ch ON _ 2018-12-18.xlsx',
                            'raw_data\\Zew ch ON _ 2018-12-18_eco.xlsx',
                          [4.86, 4.88, 4.55, 4.48, 4.2, 4.03, 3.83, 3.45, 3.17, 3.04], # fuel_means=[]
                          'load'],                                                     # charac_type ='load'
                 'tr_RME' : ['raw_data\\RME zew _ 2018-12-20.xlsx',
                            'raw_data\\RME zew _ 2018-12-20_eco.xlsx', 
                            [5.23, 5.12, 4.76, 4.6, 4.46, 4.19, 4, 3.6, 3.4, 3.17], 
                            'load']
                }

    # charac_type='load' or 'ex_speed'; rotation=1600,1800,..., 2400-rated  
    # fuel_means=[] --- we are choosing means for each mode by hands 
    def __init__(self, charac_type='load', rotation=2400, fuel_means=[]):  
        '''        !!!!!!!!!!!!!!!!!!!!!!      '''
        
        for key, val in self.raw_data.items():

In [ ]:
class Tables_manipulation:
    raw_data = {'tr_ON' : ['raw_data\\Zew ch ON _ 2018-12-18.xlsx',
                           'raw_data\\Zew ch ON _ 2018-12-18_eco.xlsx',
                          [4.86, 4.88, 4.55, 4.48, 4.2, 4.03, 3.83, 3.45, 3.17, 3.04], # fuel_means=[]
                          'ex_speed'],                                                     # charac_type ='load'
                 'tr_RME' : ['raw_data\\RME zew _ 2018-12-20.xlsx',
                            'raw_data\\RME zew _ 2018-12-20_eco.xlsx', 
                            [5.23, 5.12, 4.76, 4.6, 4.46, 4.19, 4, 3.6, 3.4, 3.17], 
                            'ex_speed']
                }
    
    
    def __init__(self):
        self.dic_dfs = {}
        self.dic_dfs['tr_ON'] = Table_main(address='raw_data\\Zew ch ON _ 2018-12-18.xlsx', 
                                           address_eco='raw_data\\Zew ch ON _ 2018-12-18_eco.xlsx', 
                                           charac_type=[4.86, 4.88, 4.55, 4.48, 4.2, 4.03, 3.83, 3.45, 3.17, 3.04], 
                                           fuel_means='ex_speed')

        return self.dic_dfs
    
    


self.dic_dfs['tr_RME'] = Table_main(address='raw_data\\RME zew _ 2018-12-20.xlsx', 
                                           address_eco='raw_data\\RME zew _ 2018-12-20_eco.xlsx', 
                                           charac_type=[5.23, 5.12, 4.76, 4.6, 4.46, 4.19, 4, 3.6, 3.4, 3.17], 
                                           fuel_means='load')

    
    def __init__(self):
        # dic to save all DataFrames processed by class Table_main DataFrames
        self.dic_dfs = {}
        for key, val in self.raw_data.items():
            self.dic_dfs[key] = Table_main(address=val[0], address_eco=val[1], charac_type=val[2], fuel_means=val[3])


In [ ]:
    ''' !!!!!  def XGBoost _ only   !!!!!!!'''
    def XGBoost_only(self): 
        
        ### start time
        self.datetime_XGB_1 = datetime.now()
        self.model_XGB = XGBRegressor(n_estimators=100, learning_rate=0.01, random_state=0)
        self.pipeline_XGB = Pipeline(steps=[('model', self.model_XGB)])
        self.lst_cross_val_score_XGB_MSE = -1 * cross_val_score(self.pipeline_XGB, self.X, self.y, cv=5, 
                                                                scoring='neg_root_mean_squared_error')
        self.lst_cross_val_score_XGB_r2 = -1 * cross_val_score(self.pipeline_XGB, self.X, self.y, cv=5, 
                                                               scoring='r2')
        self.MSE_XGB = self.lst_cross_val_score_XGB_MSE.mean()
        self.r2_XGB = self.lst_cross_val_score_XGB_r2.mean()

        ### end time
        self.datetime_XGB_2 = datetime.now()
        self.datetime_XGB = self.datetime_XGB_2 - self.datetime_XGB_1
        '''       
        self.permut_imp_XGB = PermutationImportance(self.model_XGB, random_state=1).fit(self.X, self.y)        
        self.df_permut_imp_XGB = eli5.formatters.as_dataframe.explain_weights_df(self.permut_imp_XGB, 
                                                        feature_names = self.df[self.features].columns.tolist())        
        '''

        self.dict_param['XGB: RMSE_XGB:'] = self.MSE_XGB
        self.dict_param['r2: RMSE_XGB:'] = self.r2_XGB
        self.dict_param['XGB: Time GridSearchCV worked:'] = self.datetime_XGB
        #self.dict_param['XGB: permutation_importance:'] = self.df_permut_imp_XGB
        
    